In [1]:
from snowflake.snowpark import Session

In [ ]:
# for git
connection_parameters = {
    'account': 'xxxxx',
    'user':'xxxxx',
    'password':'xxxxx'
}

new_session = Session.builder.configs(connection_parameters).create()

In [3]:
from snowflake.snowpark.functions import avg

In [ ]:
#new_session.close()

In [4]:
new_session.sql('SHOW DATABASES').show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"                      |"name"                 |"is_default"  |"is_current"  |"origin"                                            |"owner"       |"comment"                                          |"options"  |"retention_time"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2023-01-25 02:33:22.850000-08:00  |CUSTOMER_DB            |N             |N             |                                                    |ACCOUNTADMIN  |                                                   |           |1                 |
|2023-01-17 06:42:23.965000-08:0

In [5]:
new_session.sql('CREATE OR REPLACE TABLE DEMO_DB.public.sample_product_data (id INT, parent_id INT, category_id INT, name VARCHAR, serial_number VARCHAR, key INT, "3rd" INT)').collect()

[Row(status='Table SAMPLE_PRODUCT_DATA successfully created.')]

In [9]:
new_session.sql('use warehouse "COMPUTE_WH"')
new_session.sql('use role "ACCOUNTADMIN"').show()

------------------------------------
|"status"                          |
------------------------------------
|Statement executed successfully.  |
------------------------------------



In [13]:
# new_session.sql('use warehouse "COMPUTE_WH"').show()
new_session.sql('SHOW WAREHOUSES').show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"name"      |"state"    |"type"    |"size"   |"min_cluster_count"  |"max_cluster_count"  |"started_clusters"  |"running"  |"queued"  |"is_default"  |"is_current"  |"auto_suspend"  |"auto_resume"  |"available"  |"provisioning"  |"quiescing"  |"other"  |"created_on"                      |"resumed_on"                      |"updated_on"                      |"owner"       |"comment"                      

In [14]:
new_session.sql("""
INSERT INTO DEMO_DB.public.sample_product_data VALUES
(1,0,5,'Product 1','prod-1',1,10),
(2,1,5,'Product 1A','prod-1-A',1,20),
(3,1,5,'Product 1B','prod-1-B',1,30),
(4,0,10,'Product 2','prod-2',2,40),
(5,4,10,'Product 2A','prod-2-A',2,50),
(6,4,10,'Product 2B','prod-2-B',2,60),
(7, 0, 20, 'Product 3', 'prod-3', 3, 70),
(8, 7, 20, 'Product 3A', 'prod-3-A', 3, 80),
(9, 7, 20, 'Product 3B', 'prod-3-B', 3, 90),
(10, 0, 50, 'Product 4', 'prod-4', 4, 100),
(11, 10, 50, 'Product 4A', 'prod-4-A', 4, 100),
(12, 10, 50, 'Product 4B', 'prod-4-B', 4, 100)""").collect()

[Row(number of rows inserted=12)]

In [15]:
new_session.sql('SELECT count(*) from DEMO_DB.public.sample_product_data').show()

--------------
|"COUNT(*)"  |
--------------
|12          |
--------------



In [16]:
# construct a dataframe
df_table = new_session.table('DEMO_DB.public.sample_product_data')
df_table.show(5)

-------------------------------------------------------------------------------------
|"ID"  |"PARENT_ID"  |"CATEGORY_ID"  |"NAME"      |"SERIAL_NUMBER"  |"KEY"  |"3rd"  |
-------------------------------------------------------------------------------------
|1     |0            |5              |Product 1   |prod-1           |1      |10     |
|2     |1            |5              |Product 1A  |prod-1-A         |1      |20     |
|3     |1            |5              |Product 1B  |prod-1-B         |1      |30     |
|4     |0            |10             |Product 2   |prod-2           |2      |40     |
|5     |4            |10             |Product 2A  |prod-2-A         |2      |50     |
-------------------------------------------------------------------------------------



In [17]:
# create a dataframe from specified values
df1 = new_session.create_dataframe([1,2,3,4]).to_df("a")
df1.show()

-------
|"A"  |
-------
|1    |
|2    |
|3    |
|4    |
-------



In [18]:
# create a dataframe from specified values with 4 columns 'a','b','c' and'd'
df2 = new_session.create_dataframe([[1,2,3,4]],schema=['a','b','c','d'])
df2.show()

-------------------------
|"A"  |"B"  |"C"  |"D"  |
-------------------------
|1    |2    |3    |4    |
-------------------------



In [19]:
# create another dataframe with 4 columns,'a','b','c' and'd'
from snowflake.snowpark import Row
df3 = new_session.create_dataframe([Row(a=1,b=2,c=3,d=4)])
df3.show()

-------------------------
|"A"  |"B"  |"C"  |"D"  |
-------------------------
|1    |2    |3    |4    |
-------------------------



In [20]:
# create a dataframe and specify a schema
from snowflake.snowpark.types import IntegerType, StringType, StructType, StructField
schema = StructType([StructField('a', IntegerType()), StructField('b',StringType())])
df4 = new_session.create_dataframe([[1,'snow'],[3,'flake']],schema)
df4.show()

---------------
|"A"  |"B"    |
---------------
|1    |snow   |
|3    |flake  |
---------------



In [21]:
# create d ataframe from a range
# the dataframe will contain rows with values 1,3,5,7 and 9 respectively.
df_range = new_session.range(1,10,2).to_df('a')
df_range.show()

-------
|"A"  |
-------
|1    |
|3    |
|5    |
|7    |
|9    |
-------



In [ ]:
from snowflake.snowpark.types import StructType, StructField, StringType, IntegerType
# Create Dataframe from data in a stage
new_session.sql("USE DATABASE").collect()
df_json = new_session.read.json('@my_stage/data1.json')
df_catalog = new_session.read.schema(StructType([StructField('name',StringType()),StructField('age',IntegerType())])).csv('@stage/some_dir')

In [25]:
# create a dataframe from a sql query
df_sql = new_session.sql('select name from DEMO_DB.public.sample_product_data')
df_sql.show()

--------------
|"NAME"      |
--------------
|Product 1   |
|Product 1A  |
|Product 1B  |
|Product 2   |
|Product 2A  |
|Product 2B  |
|Product 3   |
|Product 3A  |
|Product 3B  |
|Product 4   |
--------------



In [26]:
# import the col function from the functions module
from snowflake.snowpark.functions import col

# create a dataframe for the rows with the id 1 in the tabl 'sample_product_data' table

df = new_session.table('DEMO_DB.public.sample_product_data').filter(col('id')==1)
df.show()

------------------------------------------------------------------------------------
|"ID"  |"PARENT_ID"  |"CATEGORY_ID"  |"NAME"     |"SERIAL_NUMBER"  |"KEY"  |"3rd"  |
------------------------------------------------------------------------------------
|1     |0            |5              |Product 1  |prod-1           |1      |10     |
------------------------------------------------------------------------------------



In [27]:
df = new_session.table('DEMO_DB.public.sample_product_data').select(col('id'),col('name'),col('serial_number'))
df.show()

---------------------------------------
|"ID"  |"NAME"      |"SERIAL_NUMBER"  |
---------------------------------------
|1     |Product 1   |prod-1           |
|2     |Product 1A  |prod-1-A         |
|3     |Product 1B  |prod-1-B         |
|4     |Product 2   |prod-2           |
|5     |Product 2A  |prod-2-A         |
|6     |Product 2B  |prod-2-B         |
|7     |Product 3   |prod-3           |
|8     |Product 3A  |prod-3-A         |
|9     |Product 3B  |prod-3-B         |
|10    |Product 4   |prod-4           |
---------------------------------------



In [28]:
df_product_info = new_session.table('DEMO_DB.public.sample_product_data')
df1 = df_product_info.select(df_product_info['id'],df_product_info['name'],df_product_info['serial_number'])
df2 = df_product_info.select(df_product_info.id,df_product_info.name,df_product_info.serial_number)
df3 = df_product_info.select('id','name','serial_number')
df1.show(2)
df2.show(2)
df3.show(2)

---------------------------------------
|"ID"  |"NAME"      |"SERIAL_NUMBER"  |
---------------------------------------
|1     |Product 1   |prod-1           |
|2     |Product 1A  |prod-1-A         |
---------------------------------------

---------------------------------------
|"ID"  |"NAME"      |"SERIAL_NUMBER"  |
---------------------------------------
|1     |Product 1   |prod-1           |
|2     |Product 1A  |prod-1-A         |
---------------------------------------

---------------------------------------
|"ID"  |"NAME"      |"SERIAL_NUMBER"  |
---------------------------------------
|1     |Product 1   |prod-1           |
|2     |Product 1A  |prod-1-A         |
---------------------------------------



In [29]:
# create two dataframe to join
df_lhs = new_session.create_dataframe([['a',1],['b',2]],schema = ['key','value1'])
df_rhs = new_session.create_dataframe([['a',3],['b',4]],schema = ['key','value2'])

# create a dataframe that joins the two dataframe on the column name 'key'
df_lhs.join(df_rhs, df_lhs.col('key')==df_rhs.col('key')).select(df_lhs['key'].as_('key'),'value1','value2').show()

-------------------------------
|"KEY"  |"VALUE1"  |"VALUE2"  |
-------------------------------
|a      |1         |3         |
|b      |2         |4         |
-------------------------------



In [30]:
# copy the dataframe if you want to do a self-join
from copy import copy
df_lhs_copied = copy(df_lhs)
df_self_joined = df_lhs.join(df_lhs_copied, (df_lhs.col('key')==df_lhs_copied.col('key')) & (df_lhs.col('value1')==df_lhs_copied.col('value1')))

In [31]:
df_lhs.join(df_rhs, df_lhs.col('key')== df_rhs.col('key')).show()

-----------------------------------------------------
|"l_rt7q_KEY"  |"VALUE1"  |"r_5ibx_KEY"  |"VALUE2"  |
-----------------------------------------------------
|a             |1         |a             |3         |
|b             |2         |b             |4         |
-----------------------------------------------------



In [32]:
df_lhs.join(df_rhs, df_lhs.col('key')== df_rhs.col('key')).select(df_lhs['key'].alias('key1'),df_rhs['key'].alias('key2'),'value1','value2').show()

-----------------------------------------
|"KEY1"  |"KEY2"  |"VALUE1"  |"VALUE2"  |
-----------------------------------------
|a       |a       |1         |3         |
|b       |b       |2         |4         |
-----------------------------------------



In [33]:
df_lhs.join(df_rhs, df_lhs.col('key')== df_rhs.col('key'), lsuffix = '_left', rsuffix='_right').show()

--------------------------------------------------
|"KEY_LEFT"  |"VALUE1"  |"KEY_RIGHT"  |"VALUE2"  |
--------------------------------------------------
|a           |1         |a            |3         |
|b           |2         |b            |4         |
--------------------------------------------------



In [34]:
from snowflake.snowpark.exceptions import SnowparkJoinException
df =  new_session.table('DEMO_DB.public.sample_product_data')
# this fails because columns named 'id' and 'parent_id' are in the left anf right dataframes in the join.
try:
    df_joined = df.join(df, col('id')== col('parent_id'))
except SnowparkJoinException as e:
    print(e.message)
    

try:
    df_joined = df.join(df, df['id']== df['parent_id'])
except SnowparkJoinException as e:
    print(e.message)

You cannot join a DataFrame with itself because the column references cannot be resolved correctly. Instead, create a copy of the DataFrame with copy.copy(), and join the DataFrame with this copy.
You cannot join a DataFrame with itself because the column references cannot be resolved correctly. Instead, create a copy of the DataFrame with copy.copy(), and join the DataFrame with this copy.


In [35]:
from copy import copy
df_lhs = new_session.table('DEMO_DB.public.sample_product_data')
df_rhs = copy(df_lhs)

df_joined = df_lhs.join(df_rhs, df_lhs.col('id')==df_rhs.col('parent_id'))
df_joined.count()

8

In [36]:
# import the col function from the functions module
df_product_info = new_session.table('DEMO_DB.public.sample_product_data').select('id','name')
df_product_info.show()

---------------------
|"ID"  |"NAME"      |
---------------------
|1     |Product 1   |
|2     |Product 1A  |
|3     |Product 1B  |
|4     |Product 2   |
|5     |Product 2A  |
|6     |Product 2B  |
|7     |Product 3   |
|8     |Product 3A  |
|9     |Product 3B  |
|10    |Product 4   |
---------------------



In [37]:
# specify the equivalent of 'where id =20' in a sql select statemnt
df_filtered = df.filter(col('id')==20)

In [38]:
df =  new_session.create_dataframe([[1,3],[2,10]],schema = ['a','b'])
# specify the equivalent of 'where a+b<10 in a sql select statement'

df_filtered = df.filter(col('a')+col('b')<10)
df_filtered.show()

-------------
|"A"  |"B"  |
-------------
|1    |3    |
-------------



In [39]:
# specify the equivalent of 'select b * 10 as c' in a sql select statement
df_selected = df.select((col('b')*10).as_('c'))
df_selected.show()

-------
|"C"  |
-------
|30   |
|100  |
-------



In [40]:
dfX = new_session.create_dataframe([[1],[2]],schema = ['a_in_X'])
dfY = new_session.create_dataframe([[1],[3]],schema = ['b_in_Y'])
# specify equivalent on 'X JOIN Y on X.a_in_X = b.b_in_Y'
df_joined = dfX.join(dfY, col('a_in_X')==col('b_in_Y')).select(dfX['a_in_X'].alias('the_joined_column'))
df_joined.show()

-----------------------
|"THE_JOINED_COLUMN"  |
-----------------------
|1                    |
-----------------------



In [41]:
# create two dataframe to join
df_lhs = new_session.create_dataframe([['a',1],['b',2]],schema = ['key','value1'])
df_rhs = new_session.create_dataframe([['a',3],['b',4]],schema = ['key','value2'])

# create a dataframe that joins the two dataframe on the column name 'key'
df_lhs.join(df_rhs, df_lhs.col('key')==df_rhs.col('key')).select(df_lhs['key'].as_('key'),df_lhs.col('value1').as_('L'),df_rhs.col('value2').as_('R')).show()

---------------------
|"KEY"  |"L"  |"R"  |
---------------------
|a      |1    |3    |
|b      |2    |4    |
---------------------



In [73]:
# new_session.sql('use database DEMO_DB')

In [42]:

new_session.sql("""
create or replace table DEMO_DB.public."10tablename"
(
id123 varchar,
"3rdID" varchar,
"id with space" varchar)""").collect()



[Row(status='Table 10tablename successfully created.')]

In [48]:
new_session.sql("""insert into DEMO_DB.public."10tablename" (id123, "3rdID","id with space") values('a','b','c')""").collect()
df = new_session.table('DEMO_DB.public."10tablename"')
df.show()

---------------------------------------
|"ID123"  |"3rdID"  |"id with space"  |
---------------------------------------
|a        |b        |c                |
|a        |b        |c                |
---------------------------------------



In [49]:
df.select(col('id123')).collect()
df.select(col("ID123")).collect()

[Row(ID123='a'), Row(ID123='a')]

In [45]:
# df = new_session.table("\"10tablename\"")
# df = new_session.table('"10tablename"')

In [50]:
from snowflake.snowpark.functions import col,lit
df_filtered = df.filter(col('num_items')>lit(5))

In [51]:
from snowflake.snowpark.types import DecimalType
decimal_vale = lit(0.05).cast(DecimalType(5,2))

In [52]:
df_product_info = new_session.table('DEMO_DB.public.sample_product_data').filter(col('id')==1).select(col('name'),col('serial_number'))
df_product_info.show()

-------------------------------
|"NAME"     |"SERIAL_NUMBER"  |
-------------------------------
|Product 1  |prod-1           |
-------------------------------



In [56]:
table_schema = new_session.table('DEMO_DB.public.sample_product_data').schema
table_schema

StructType([StructField('ID', LongType(), nullable=True), StructField('PARENT_ID', LongType(), nullable=True), StructField('CATEGORY_ID', LongType(), nullable=True), StructField('NAME', StringType(), nullable=True), StructField('SERIAL_NUMBER', StringType(), nullable=True), StructField('KEY', LongType(), nullable=True), StructField('"3rd"', LongType(), nullable=True)])

In [58]:
# create dataframe containing the 'id' and '3rd' columns
df_selected_columns = new_session.table('DEMO_DB.public.sample_product_data').select(col('id'),col("3rd"))
df_selected_columns.schema.names

['ID', '"3rd"']

In [59]:
df = new_session.table('DEMO_DB.public.sample_product_data').select(col('id'),col('name'))
results = df.collect()

In [60]:
df_products = new_session.table('DEMO_DB.public.sample_product_data')
print(df_products.count())

12


In [62]:
# create a dataframe for the 'sample_product_data' table
df_products = new_session.table('DEMO_DB.public.sample_product_data')
df_products.show(20)

-------------------------------------------------------------------------------------
|"ID"  |"PARENT_ID"  |"CATEGORY_ID"  |"NAME"      |"SERIAL_NUMBER"  |"KEY"  |"3rd"  |
-------------------------------------------------------------------------------------
|1     |0            |5              |Product 1   |prod-1           |1      |10     |
|2     |1            |5              |Product 1A  |prod-1-A         |1      |20     |
|3     |1            |5              |Product 1B  |prod-1-B         |1      |30     |
|4     |0            |10             |Product 2   |prod-2           |2      |40     |
|5     |4            |10             |Product 2A  |prod-2-A         |2      |50     |
|6     |4            |10             |Product 2B  |prod-2-B         |2      |60     |
|7     |0            |20             |Product 3   |prod-3           |3      |70     |
|8     |7            |20             |Product 3A  |prod-3-A         |3      |80     |
|9     |7            |20             |Product 3B  |pro

In [63]:
python_df = new_session.create_dataframe(['a','b','c'])
pandas_df = python_df.to_pandas()

In [65]:
df.write.mode('overwrite').save_as_table('DEMO_DB.public.table1')

In [ ]:
import os
database = os.environ["DEMO_DB"]
schema = os.environ["public"]
view_name = "my_view"
df.create_or_replace_view(f"{database}.{schema}.{view_name}")

In [78]:
from snowflake.snowpark.types import *
schema_for_data_file = StructType([
    StructField("id",StringType()),
    StructField("name",StringType())
])

In [79]:
df_reader = new_session.read.schema(schema_for_data_file)

In [80]:
df_reader = df_reader.option('field_delimiter',';').option('COMPRESSION','NONE')

In [ ]:
new_session.sql("USE DATABASE").collect() 
df = df_reader.csv('@s3_ts_stage/emails/data_0_0_0.csv')

In [ ]:
# import the sql_expr function from the functions module
from snowflake.snowpark.functions import sql_expr
df = new_session.read.json('@my_stage').select(sql_expr('$1:color'))

In [83]:
new_session.sql("create or replace temp stage DEMO_DB.public.my_stage").collect()

[Row(status='Stage area MY_STAGE successfully created.')]

In [85]:
stage_files_df = new_session.sql('ls @DEMO_DB.public.my_stage').collect()

In [86]:
new_session.sql("alter warehouse if exists my_warehouse resume if suspended").collect()

[Row(status='Statement executed successfully.')]

In [88]:
new_session.sql('copy into DEMO_DB.public.sample_product_data from @DEMO_DB.public.my_stage file_format =(type = csv)').collect()

[Row(status='Copy executed with 0 files processed.')]

In [89]:
new_session.close()